## phase 1. Data Collection

### 1. Import Libraries

In [1]:
import pandas as pd
import requests
import time
from sklearn.preprocessing import MinMaxScaler

### 2. Load Wallet Addresses

In [2]:
# Load wallet list
df = pd.read_csv("wallet_id.csv")
df.columns = df.columns.str.strip()

wallets = df["wallet_id"].dropna().tolist()
print("Total wallets loaded:", len(wallets))


Total wallets loaded: 103


### 3. Define API Key

In [3]:
# Covalent API key
api_key = "cqt_rQgXvWBXy6Km7myBRQf8FcgR6pqF"


### 4. Define Function to Fetch Transactions

In [4]:
import requests
import time

def fetch_wallet_transactions(wallet_address, retries=3):
    api_key = "cqt_rQgXvWBXy6Km7myBRQf8FcgR6pqF"  
    url = f"https://api.covalenthq.com/v1/1/address/{wallet_address}/transactions_v2/?key={api_key}"
    
    for attempt in range(retries):
        print(f"Attempt {attempt+1} for {wallet_address}") 
        
        try:
            response = requests.get(url)
            print(f"Status Code: {response.status_code}")  
            
            if response.status_code == 200:
                data = response.json()
                txs = data.get("data", {}).get("items", [])
                
                print(f"Transactions fetched: {len(txs)}")
                return txs
            else:
                print(f"Error {response.status_code} for wallet {wallet_address}")
                return []
        
        except Exception as e:
            print(f"Exception on attempt {attempt+1}: {e}")
            time.sleep(1)
    
    print("❗ All attempts failed.")
    return []


In [5]:
test_wallet = "0x0039f22efb07a647557c7c5d17854cfd6d489ef3"
transactions = fetch_wallet_transactions(test_wallet)
print(f"Total transactions returned: {len(transactions)}")


Attempt 1 for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Status Code: 200
Transactions fetched: 100
Total transactions returned: 100


### 5. Loop Over Wallets to Collect Transactions

In [6]:
all_transactions = []

for i, wallet in enumerate(wallets):
    print(f"Processing wallet {i+1}/{len(wallets)}: {wallet}")
    txs = fetch_wallet_transactions(wallet)
    
    for tx in txs:
        all_transactions.append({
            "wallet_address": wallet,
            "tx_hash": tx.get("tx_hash"),
            "block_signed_at": tx.get("block_signed_at"),
            "value": tx.get("value"),
            "from_address": tx.get("from_address"),
            "to_address": tx.get("to_address"),
            "gas_spent": tx.get("gas_spent"),
            "successful": tx.get("successful"),
        })
    
    time.sleep(1)  # respect API rate limit


Processing wallet 1/103: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Attempt 1 for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Status Code: 200
Transactions fetched: 100
Processing wallet 2/103: 0x06b51c6882b27cb05e712185531c1f74996dd988
Attempt 1 for 0x06b51c6882b27cb05e712185531c1f74996dd988
Status Code: 200
Transactions fetched: 5
Processing wallet 3/103: 0x0795732aacc448030ef374374eaae57d2965c16c
Attempt 1 for 0x0795732aacc448030ef374374eaae57d2965c16c
Status Code: 200
Transactions fetched: 4
Processing wallet 4/103: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Attempt 1 for 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Status Code: 200
Transactions fetched: 80
Processing wallet 5/103: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Attempt 1 for 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Status Code: 200
Transactions fetched: 5
Processing wallet 6/103: 0x104ae61d8d487ad689969a17807ddc338b445416
Attempt 1 for 0x104ae61d8d487ad689969a17807ddc338b445416
Status Code: 200
Transactions fetched: 6
P

### 6. Save Raw Transactions to CSV

In [7]:
tx_df = pd.DataFrame(all_transactions)
tx_df.to_csv("wallet_transactions.csv", index=False)
print("Saved wallet_transactions.csv with", len(tx_df), "rows")


Saved wallet_transactions.csv with 2823 rows


### 7.Load CSV

In [8]:
df=pd.read_csv("wallet_transactions.csv")

In [9]:
df

,wallet_address,tx_hash,block_signed_at,value,from_address,to_address,gas_spent,successful
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x98703fb4a7c6804d82e98f009ecc0e089abd53de9469...,2025-06-16T21:15:11Z,0,0xc6b602de080fc9ac9d96a431b2d749d38e77cbbc,0x13173761e24c3708495b1dd314920f67f97011d0,55815,True
1,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x0b8e89905717b9babd69aa7db6af0afb83331e9b8041...,2025-04-27T21:09:35Z,0,0xfd0ce77ca6521e3cbbcfb07398cca8ec2648115d,NaN,5264347,True
2,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x5745f6d2e9397ed8ceb7120f42761e1f04643e132e94...,2025-03-24T23:27:47Z,0,0xc07aff7c831105bacaf2fb4d9506332112eee2a1,NaN,5226097,True
3,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x35cd404392ac9b7786bee78e4bfb2773cbb576b9c3dc...,2025-02-23T19:08:59Z,0,0xad38352c725c6768c11a9a71bc7d921a9355d56c,NaN,5213387,True
4,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x51a4f04abaa498409b72d83b4365440f0738ac5bcca1...,2025-02-11T03:05:11Z,0,0x1e6971036be043c0e2457fe3f6dbed9f73354c6b,NaN,5244910,True
...,...,...,...,...,...,...,...,...
2818,0xf80a8b9cfff0febf49914c269fb8aead4a22f847,0x55030001bba1945f08497ff31ea3e836ba48b68b4726...,2020-07-29T06:16:50Z,0,0xfdcab1b4e6292ddd540a3c535f36d32d879aa78a,0x1c647c67c1aad0ebff25500f3b3084d3d2872eba,535112,True
2819,0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac,0xe76b49f4a25d49d8e5279e905ca683488f1a21313e57...,2020-10-21T18:51:46Z,0,0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac,0x39aa39c021dfbae8fac545936693ac917d5e7563,202784,True
2820,0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac,0x5a10a092184342b7e2132f7a60868a735c78b85445ed...,2020-10-21T18:51:31Z,0,0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,49811,True
2821,0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac,0xf2135f7e2a3490c5706f36def69926a5b3ae71140bab...,2020-10-21T11:57:13Z,0,0x73008d40648d3ccd191d9af6889159c15089095d,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,42369,True


## phase 2.Feature Engineering

In [10]:
import pandas as pd

# Load your collected transaction data
df = pd.read_csv("wallet_transactions.csv")

# Clean column names if needed
df.columns = df.columns.str.strip()

# Convert timestamp to datetime
df['block_signed_at'] = pd.to_datetime(df['block_signed_at'])

# Calculate USD value per transaction (optional, here just using 'value')
df['value'] = pd.to_numeric(df['value'], errors='coerce').fillna(0)

# Aggregate features per wallet
features = df.groupby('wallet_address').agg(
    total_transactions=('tx_hash', 'count'),
    total_value=('value', 'sum'),
    avg_value=('value', 'mean'),
    gas_spent=('gas_spent', 'sum'),
    unique_to_addresses=('to_address', pd.Series.nunique),
    unique_from_addresses=('from_address', pd.Series.nunique),
    first_tx=('block_signed_at', 'min'),
    last_tx=('block_signed_at', 'max'),
    success_rate=('successful', lambda x: x.sum() / len(x))
).reset_index()

features['activity_days'] = (features['last_tx'] - features['first_tx']).dt.days + 1

# Save to CSV for next step
features.to_csv("wallet_features.csv", index=False)
print("Step 2 done: wallet_features.csv generated.")


Step 2 done: wallet_features.csv generated.


## phase 3: Assigning Wallet Risk Scores (0–1000)


In [11]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

features = pd.read_csv("wallet_features.csv")

score_columns = ['total_transactions', 'total_value', 'avg_value', 'gas_spent', 'success_rate', 'activity_days']

features[score_columns] = features[score_columns].fillna(0)

# Normalize the selected features to range [0, 1]
scaler = MinMaxScaler()
normalized = scaler.fit_transform(features[score_columns])
features_normalized = pd.DataFrame(normalized, columns=score_columns)

features['score'] = (features_normalized.mean(axis=1) * 1000).astype(int)

final_scores = features[['wallet_address', 'score']]
final_scores.columns = ['wallet_id', 'score']

final_scores.to_csv("wallet_scores.csv", index=False)
print(" Step 3 done: wallet_scores.csv generated.")


 Step 3 done: wallet_scores.csv generated.


In [12]:
df = pd.read_csv("wallet_scores.csv")
print(df.head())


                                    wallet_id  score
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3    505
1  0x06b51c6882b27cb05e712185531c1f74996dd988    170
2  0x0795732aacc448030ef374374eaae57d2965c16c    168
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9    542
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae    173
